In [1]:
META_PATH = '/Users/jakubmalczak/UNI/INŻ/SequentialRecommendation/recbole/data/dataset/Amazon_Sports_and_Outdoors/meta_Sports_and_Outdoors_2023.jsonl'
SAVE_PATH = '/Users/jakubmalczak/UNI/INŻ/SequentialRecommendation/recbole/data/dataset/Amazon_Sports_and_Outdoors/Amazon_Sports_and_Outdoors_2023.item'

### Reading JSONL file

In [42]:
import pandas as pd
import json

def safe_float(x):
    try:
        return float(x)
    except (TypeError, ValueError):
        return float('nan')

data = []
with open(META_PATH, 'r', encoding='utf-8') as f:
    for line in f:
        record = json.loads(line)
        best_sellers = record.get("details", {}).get("Best Sellers Rank", {})
        max_key, max_value = None, float('nan')
        if best_sellers:
            max_key = max(best_sellers, key=best_sellers.get)
            max_value = best_sellers[max_key]
        data.append(
            {
                "item_id:token": record.get("parent_asin"),
                "title:token": record.get("title"),
                "price:float": safe_float(record.get("price")),
                # store 35109 times None vs. details -> Brand Name 572787 times None
                "brand:token": record.get("store"),
                "categories:token_seq": '\'' + "\', \'".join(record.get("categories")) + '\'' if record.get("categories") else '',
                "sales_type:token": max_key,
                "sales_rank:float": max_value,
            }
        )

df = pd.DataFrame(data)
df.head(5)

,item_id:token,title:token,price:float,brand:token,categories:token_seq,sales_type:token,sales_rank:float
0,B01HDXC8AG,Sure-Grip Zombie Wheels Low 59mm 4 Pack,55.00,Sure-Grip,"'Sports & Outdoors', 'Sports', 'Skates, Skateb...",Sports & Outdoors,295175.0
1,B07R5BQ4YD,USGI Wet Weather Bag (Fоur Paсk),NaN,USGI,"'Sports & Outdoors', 'Sports', 'Boating & Sail...",Sports & Outdoors,962400.0
2,B003K8GZ7G,NHL San Jose Sharks Team Logo Post Earrings,18.99,Aminco,"'Sports & Outdoors', 'Fan Shop', 'Jewelry & Wa...",Sports & Outdoors,721263.0
3,B08GC4GBWB,Bont Skates - Prostar Purple Suede Professiona...,209.00,Bont,"'Sports & Outdoors', 'Sports', 'Skates, Skateb...",Sports & Outdoors,213685.0
4,B07BYV947H,Team Golf Alamaba Crimson Tide Embroidered Tow...,NaN,Team Golf,"'Sports & Outdoors', 'Fan Shop', 'Sports Equip...",Sports & Outdoors,1621084.0


### Validate data

In [43]:
df.isna().sum()

item_id:token                 0
title:token                   0
price:float             1102862
brand:token               35109
categories:token_seq          0
sales_type:token         963183
sales_rank:float         963183
dtype: int64

In [44]:
string_cols = df.select_dtypes(include="object").columns
for col in string_cols:
    print(col, "=> puste stringi:", (df[col] == "").sum())

item_id:token => puste stringi: 0
title:token => puste stringi: 112
brand:token => puste stringi: 0
categories:token_seq => puste stringi: 89830
sales_type:token => puste stringi: 0


In [45]:
df[["price:float", "sales_rank:float"]].describe().map(lambda x: f"{x:,.0f}")

,price:float,sales_rank:float
count,"484,559","624,238"
mean,57,"735,504"
std,162,"614,582"
min,0,1
25%,13,"267,578"
50%,24,"589,380"
75%,50,"1,054,199"
max,"22,000","13,516,189"


### Save to .item file

In [46]:
df.to_csv(SAVE_PATH, sep='\t', index=False, na_rep='')